# Notebook for using modelvalidation and modeltune to validate supervised predictive models
This is a template Jupyter notebook for using modelvalidation() and modeltune() functions to optimise and supervised predictive models.

For each code cell, there is an indicator says "In [ ]". This shows status of the cell. If there is noting inside "[ ]" it means this cell has not been evaluated yet; if there is a "\*" inside "[ ]" it means that this cell is currently being evaluated and the R kernel is busy (i.e. you should wait); if there is a number inside "[ ]", e.g. "[1]", it means that the current cell has already been evaluated. Place cursor anywhere in the cell (by mouse clicking) or highlight the cell by pressing up/down arrow key and press shift-enter to run it.

__It is advisable to reset this notebook before you run it to clear remainants from previous analysis. Click menu "Kernel" -> "Restart Kernel and Clear all Outputs".__

# 1. Load required libraries and functions

__Note: in house pls functions (loaded from modelling_tools.R) are used in this notebook. Validation functions are stored in validation.R, can be loaded separately if use other modelling functions__

In [13]:
library(caret)
library(permute)
library(pracma)
source("https://raw.githubusercontent.com/Biospec/Cluster_toolbox_for_R/master/modelling_tools.R")
source("https://raw.githubusercontent.com/Biospec/Cluster_toolbox_for_R/master/validation.R")

# 2. Set work directory to where the data are sotred and load the data.
In this example, data matrix is stored in data_reg.csv and the concentrations are stored in conc_reg.csv. These two files are stored under folder "~/Dropbox/biospec/". Change accordingly to fit your own data.

In [2]:
setwd("~/Dropbox/biospec")
x <- read.csv("data_reg.csv", header = FALSE)
y <- read.csv("conc_reg.csv", header = FALSE)

# 3. Model selection and validation

## 3.1 Set parameters for modeltune():

train_fun: the name of model training function, e.g. pls for partial least squares.

pred_fun: the prediction function which apply test data to the trained model, e.g. plspred.

eval_type = "R", the type of prediction, can be "R" for regression and "C" for classification, set to "R" by default.

X, Y: the name of variables of the data matrix X and the label vector/matrix Y.

model_parameters: the range of model parameters to be tuned, e.g. for PLS the number of PLS component

__The example line below train a pls regression model on the data stored in x with labels in y, the number of PLS components to be tested is set from 1 to 20.__

In [3]:
plsr_tuned <- modeltune(train_fun = pls, pred_fun = plspred, X = x, Y = y, 
                        model_parameters = list(lv = 1:20)) 
#the model parameters are to be stored in a paired list with the name and possible 
#choices, PLS model only require one parameter: the number of latent variables (LV)
#1:20 means testing the model with 1 to 20 latent variables and chose one with lowest
#CV error. 


In [19]:
# Examine the results
print(plsr_tuned$opt_parameters) # Optimal model parameter

$lv
[1] 10

$Xmc
[1] FALSE

$Ymc
[1] FALSE

$Xscale
[1] FALSE

attr(,"out.attrs")
attr(,"out.attrs")$dim
    lv    Xmc    Ymc Xscale 
    20      2      2      2 

attr(,"out.attrs")$dimnames
attr(,"out.attrs")$dimnames$lv
 [1] "lv= 1" "lv= 2" "lv= 3" "lv= 4" "lv= 5" "lv= 6" "lv= 7" "lv= 8" "lv= 9"
[10] "lv=10" "lv=11" "lv=12" "lv=13" "lv=14" "lv=15" "lv=16" "lv=17" "lv=18"
[19] "lv=19" "lv=20"

attr(,"out.attrs")$dimnames$Xmc
[1] "Xmc=TRUE"  "Xmc=FALSE"

attr(,"out.attrs")$dimnames$Ymc
[1] "Ymc=TRUE"  "Ymc=FALSE"

attr(,"out.attrs")$dimnames$Xscale
[1] "Xscale=TRUE"  "Xscale=FALSE"




In [20]:
print(plsr_tuned$opt_results) # Optimal predictive accuracy

      r2cv     rmsecv 
 0.9549312 46.3795992 


__A more complicated tunning: including chosing whether to perform mean centring on X and Y block, whether to perform autoscaling on X block, pay attention to additional parameter choices added to model_parameters__

In [5]:
plsr_tuned <- modeltune(train_fun = pls, pred_fun = plspred, X = x, Y = y,
                        model_parameters = list(lv = 1:20, Xmc = c(TRUE, FALSE),
                                                Ymc = c(TRUE, FALSE), Xscale = 
                                                  c(TRUE, FALSE)))

In [6]:
print(plsr_tuned$opt_parameters)
print(plsr_tuned$opt_results)

$lv
[1] 10

$Xmc
[1] FALSE

$Ymc
[1] FALSE

$Xscale
[1] FALSE

attr(,"out.attrs")
attr(,"out.attrs")$dim
    lv    Xmc    Ymc Xscale 
    20      2      2      2 

attr(,"out.attrs")$dimnames
attr(,"out.attrs")$dimnames$lv
 [1] "lv= 1" "lv= 2" "lv= 3" "lv= 4" "lv= 5" "lv= 6" "lv= 7" "lv= 8" "lv= 9"
[10] "lv=10" "lv=11" "lv=12" "lv=13" "lv=14" "lv=15" "lv=16" "lv=17" "lv=18"
[19] "lv=19" "lv=20"

attr(,"out.attrs")$dimnames$Xmc
[1] "Xmc=TRUE"  "Xmc=FALSE"

attr(,"out.attrs")$dimnames$Ymc
[1] "Ymc=TRUE"  "Ymc=FALSE"

attr(,"out.attrs")$dimnames$Xscale
[1] "Xscale=TRUE"  "Xscale=FALSE"


      r2cv     rmsecv 
 0.9549312 46.3795992 


__Now optimising SVM-regression from e1071 package__

This example try SVR with linear kernel, with 10 different settings to each kernel parameter respectively (a lot of combinations, will take a while to finish.).

In [12]:
library(e1071)
svm_tuned <- modeltune(train_fun = svm, pred_fun = predict, X = x, Y = y,
                       model_parameters = list(type = 'eps-regression', kernel = 'linear', 
                                               cost = logspace(-3,2,10), epsilon = logspace(-6,0,10)))

In [14]:
print(svm_tuned$opt_parameters)

$type
[1] eps-regression
Levels: eps-regression

$kernel
[1] linear
Levels: linear

$cost
[1] 0.5994843

$epsilon
[1] 0.01

attr(,"out.attrs")
attr(,"out.attrs")$dim
   type  kernel    cost epsilon 
      1       1      10      10 

attr(,"out.attrs")$dimnames
attr(,"out.attrs")$dimnames$type
[1] "type=eps-regression"

attr(,"out.attrs")$dimnames$kernel
[1] "kernel=linear"

attr(,"out.attrs")$dimnames$cost
 [1] "cost=1.000000e-03" "cost=3.593814e-03" "cost=1.291550e-02"
 [4] "cost=4.641589e-02" "cost=1.668101e-01" "cost=5.994843e-01"
 [7] "cost=2.154435e+00" "cost=7.742637e+00" "cost=2.782559e+01"
[10] "cost=1.000000e+02"

attr(,"out.attrs")$dimnames$epsilon
 [1] "epsilon=1.000000e-06" "epsilon=4.641589e-06" "epsilon=2.154435e-05"
 [4] "epsilon=1.000000e-04" "epsilon=4.641589e-04" "epsilon=2.154435e-03"
 [7] "epsilon=1.000000e-02" "epsilon=4.641589e-02" "epsilon=2.154435e-01"
[10] "epsilon=1.000000e+00"




In [15]:
print(svm_tuned$opt_results)        

      r2cv     rmsecv 
 0.9037327 67.7842106 


## 3.2 Model validation

modelvalidation() has a few additional parameters which have been set by a set of default values as follows:

    resample_method = "boots", select the resampling method, can be either "boots" or "crossval" for bootstrapping and cross-validation, "boots" by default.

    k= 1000, a positive number, the number of iterations of bootstrapping or the number of folds of cross-validation. If resample_method = "crossval" and k >= no. of samples then leave-one-out crossvalidation will be performed.

    cv_perm = FALSE, control whether to perform a permutation of the orders of samples before crossvalidation (to make cross-validation less deterministic).

    perm_test = FALSE, control whether to perform a paired permutation test and assess the significance level of the model aganst NULL data.

In [16]:
plsr_validated <- modelvalidation(train_fun = pls, pred_fun = plspred, eval_type = 'R',
                                   resample_method = 'boots', k= 100, X = x, Y = y, 
                                   model_parameters = list(lv = 1:20)) #100 bootstrapping validations

In [17]:
print(median(plsr_validated$RMSEP))

[1] 57.87082


In [18]:
print(median(plsr_validated$R2P))

[1] 0.8706725
